## Enforcing Amazon Bedrock Guardrails with AWS IAM conditions

In this example, we demonstrate how to enforce the use of a given Guardrail for invoking models in Amazon Bedrock. This helps ensuring the models are always called with the your desired protections for prompts and/or responses, aligned with your company's policies and responsible AI.

For more information, check the documentation here: https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-permissions.html#guardrails-permissions-id


### Setup

Let's start by installing or upgrading Boto3, and setting up an Amazon STS client for double-checking our account number.

In [ ]:
### Uncomment and run the first time...
#!pip install boto3 --upgrade

In [ ]:
import boto3
import json

region = 'us-west-2' ### Replace with your desired region

session = boto3.Session(region_name=region, profile_name='default') ### Replace with your desired profile name
sts_client = session.client('sts', region_name=region)
account_id = sts_client.get_caller_identity()["Account"]
print(f'Account ID: {account_id}')

### Pre-requisites

**IMPORTANT:** Make sure you have an Amazon Bedrock Guardrail created as per the steps here: https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-create.html

Take note of your Guardrail ID and replace in the cell below.

In [ ]:
guardrail_id = "[guardrailID]" ### Replace with the ID of your guardrail

### Policy definitions

The cell below contains an example policy for only allowing invocations when the given guardrail is included in the request.

Adjust this policy to your needs as per the documentation here: https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-permissions.html#guardrails-permissions-id

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "InvokeFoundationModelStatement1",
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream"
            ],
            "Resource": [
                "arn:aws:bedrock:{region}::foundation-model/*"
            ],
            "Condition": {
                "StringLike": {
                    "bedrock:GuardrailIdentifier": "arn:aws:bedrock:{region}:{account_id}:guardrail/{guardrail_id}*"
                }
            }
        },
        {
            "Sid": "ApplyGuardrail",
            "Effect": "Allow",
            "Action": [
                "bedrock:ApplyGuardrail"
            ],
            "Resource": [
                "arn:aws:bedrock:{region}:{account_id}:guardrail/{guardrail_id}"
            ]
        }
    ]
}

#Replace the placeholders for region and account ID in the policy...
policy_str = json.dumps(policy)
policy_str = policy_str.replace('{region}', region)
policy_str = policy_str.replace('{account_id}', account_id)
policy_str = policy_str.replace('{guardrail_id}', guardrail_id)
policy = json.loads(policy_str)
print(json.dumps(policy, indent=2))

We'll now create an AWS IAM role containing our policy. Alternatively, modify an existing role for attaching the policy.

In [ ]:
def create_bedrock_role(bedrock_role_name):
    iam_client = session.client('iam')
    
    # Trust policy for AWS services
    trust_policy = {
    	"Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": [
                        "bedrock.amazonaws.com",
                        "sagemaker.amazonaws.com"
                    ]
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    # Bedrock access policy
    bedrock_policy = json.dumps(policy)
    
    try:
        # Create Bedrock role
        bedrock_role = iam_client.create_role(
            RoleName=bedrock_role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy)
        )
        # Create and attach Bedrock policy
        bedrock_policy_name = f"{bedrock_role_name}Policy"
        iam_client.create_policy(
            PolicyName=bedrock_policy_name,
            PolicyDocument=json.dumps(bedrock_policy)
        )
        iam_client.attach_role_policy(
            RoleName=bedrock_role_name,
            PolicyArn=f"arn:aws:iam::{sts_client.get_caller_identity()['Account']}:policy/{bedrock_policy_name}"
        )
        print(f"Created Bedrock execution role: {bedrock_role['Role']['Arn']}")
        return bedrock_role['Role']['Arn']
        
    except Exception as e:
        print(f"Error creating Bedrock Guardrails role: {str(e)}")
        raise

In [ ]:
### Run this if you want to create a new role...
bedrock_execution_role = create_bedrock_role('BedrockGuardrailsRole')

### Or, uncomment and set this if you already have your own role and have attached the policy to it...
#bedrock_execution_role = "[your-role-arn]" ### Replace with your existing role ARN

### Setup Bedrock client with the role created

We're ready to test our policy. For this, we'll setup the Bedrock client with the role we've defined above.

In [19]:
credentials = boto3.client('sts').assume_role(
    RoleArn=bedrock_execution_role,
    RoleSessionName='assume-role')["Credentials"]

bedrock_session = boto3.session.Session(
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken'])

bedrock = bedrock_session.client(
    service_name='bedrock-runtime',
    region_name=region
)

### Test the enforcement of the Guardrail

#### 1. Invoking a model without passing the required Guardrail

We'll start by invoking a model in Amazon Bedrock using the Converse API, but we'll not attach any Guardrail in the invocation.

**NOTE:** This should return an **AccessDenied** error.

In [ ]:
### Without Guardrails...

response = bedrock.converse(
        modelId="anthropic.claude-3-5-haiku-20241022-v1:0",
        messages=[{"role": "user", "content": [{"text": "Hello, how are you today?"}]}],
        system=[{"text": "You're a helpful assistant"}],
        inferenceConfig={
            "maxTokens": 8000,
            "temperature": 0,
        },
)

print(json.dumps(response, indent=4, default=str, ensure_ascii=False))

#### 2. Invoking a model with the required Guardrail

We'll now invoke with the Guardrail ID in the request.

**NOTE:** This should allow the call to go through.

In [ ]:
#With Guardrails...

response = bedrock.converse(
        modelId="anthropic.claude-3-5-haiku-20241022-v1:0",
        messages=[{"role": "user", "content": [{"text": "Hello, how are you today?"}]}],
        system=[{"text": "You're a helpful assistant"}],
        inferenceConfig={
            "maxTokens": 8000,
            "temperature": 0,
        },
        guardrailConfig={
            "guardrailIdentifier": f"arn:aws:bedrock:{region}:{account_id}:guardrail/{guardrail_id}", ### This should map to your guardrail ARN
            "guardrailVersion": "DRAFT", ### Replace with your desired guardrail version
        },
)

print(json.dumps(response, indent=4, default=str, ensure_ascii=False))

For more information, read the blog post and documentation supporting this feature.